## kaggle json 파일을 업로드

In [11]:
#!cat /root/.kaggle/kaggle.json

In [9]:
KAGGLE_UPLOAD = False

if KAGGLE_UPLOAD:
    # /content/ 폴더에 kaggle.json을 업로드 (매번 colab노트북 생성시 반복)
    from google.colab import files
    files.upload()
else:
    kaggle_username = 'ventureparker'
    from google.colab import userdata
    kaggle_key = userdata.get(kaggle_username)

### Kaggle.json파일 생성

In [12]:
import json
import os

def create_kaggle_json(username, key, file_path):
    # 데이터 생성
    kaggle_data = {
        "username": username,
        "key": key
    }

    # JSON 파일로 저장
    with open(file_path, 'w') as json_file:
        json.dump(kaggle_data, json_file)

    print(f"kaggle.json 파일이 '{os.path.abspath(file_path)}' 경로에 생성되었습니다.")



# kaggle.json 파일 생성 함수 호출
if KAGGLE_UPLOAD==False:
    # 사용자 정보 입력
    key = kaggle_key  # 여기에 key값을 입력하세요
    filename = 'kaggle.json'
    create_kaggle_json(kaggle_username, key, filename)

kaggle.json 파일이 '/content/kaggle.json' 경로에 생성되었습니다.


In [13]:
!cat kaggle.json

{"username": "ventureparker", "key": "dd42a6316bbc050370e11511e7a91e9e"}

In [5]:
import os
import shutil

os.makedirs('/root/.kaggle', exist_ok=True)
path = os.getcwd()

# 파일을 복사하는데, 이미 복사된 경우 복사하지 않는다.
if not os.path.exists(os.path.join('/root/.kaggle','kaggle.json')):
    src = os.path.join(path,'kaggle.json')
    dst = os.path.join('/root/.kaggle', 'kaggle.json')
    shutil.copy(src,dst)
    print('파일 복사 완료')

파일 복사 완료


In [6]:
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c dogs-vs-cats

 99% 808M/812M [00:21<00:00, 39.2MB/s]
100% 812M/812M [00:21<00:00, 39.2MB/s]


In [ ]:
# 압축파일 풀기 :하위 폴더 data에 풀기
!unzip -qq dogs-vs-cats.zip -d data

In [ ]:
import zipfile

data = os.path.join(path,'data')
train_data = os.path.join(data,'train')
test_data = os.path.join(data,'test1')

zip_file = zipfile.ZipFile(os.path.join(data,'train.zip'))
zip_file.extractall(path=data)
zip_file.close()

zip_file = zipfile.ZipFile(os.path.join(data,'test1.zip'))
zip_file.extractall(path=data)
zip_file.close()

In [ ]:
# 데이터의 갯수
trainFiles = os.listdir(train_data)
testFiles = os.listdir(test_data)
# train : 12500개 cat, 12500개 dog
print(len(trainFiles))
# test : 레이블이 없는 이미지 12500개
print(len(testFiles))

25000
12500


In [ ]:
!rm -rf sdata

In [ ]:
sdata = 'sdata'
tain_sdata = os.path.join(sdata,'train')
valid_sdata = os.path.join(sdata,'valid')
test_sdata = os.path.join(sdata,'test')

os.makedirs(tain_sdata, exist_ok=True)
os.makedirs(valid_sdata, exist_ok=True)
os.makedirs(test_sdata, exist_ok=True)

train_sdata_dog = os.path.join(tain_sdata,'dog')
train_sdata_cat = os.path.join(tain_sdata,'cat')
os.makedirs(train_sdata_dog, exist_ok=True)
os.makedirs(train_sdata_cat, exist_ok=True)

valid_sdata_dog = os.path.join(valid_sdata,'dog')
valid_sdata_cat = os.path.join(valid_sdata,'cat')
os.makedirs(valid_sdata_dog, exist_ok=True)
os.makedirs(valid_sdata_cat, exist_ok=True)

In [ ]:
# 한 클래스의 이미지 갯수 * 2
train_num = 1000
valid_num = 500
test_num  = 1000

train_range = [0, train_num-1]
valid_range = [train_num, train_num + valid_num -1]
test_range  = [train_num + valid_num, train_num + valid_num + test_num-1]

print(train_range)
print(valid_range)
print(test_range)

[0, 999]
[1000, 1499]
[1500, 2499]


In [ ]:
import os
import shutil

# 데이터셋 디렉토리 경로 설정 (데이터셋이 저장된 경로로 수정하세요)
dataset_dir = 'data/train'

# 새로운 train, valid, test 디렉토리 생성 경로
base_dir = 'sdata'

# 클래스 목록
classes = ['dog', 'cat']

# 폴더 경로 생성
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

# 폴더 생성 함수
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# train, valid, test 폴더 및 각각의 클래스 폴더 생성
for cls in classes:
    create_dir(os.path.join(train_dir, cls))
    create_dir(os.path.join(valid_dir, cls))

# 이미지 복사 함수
def copy_images(start_idx, end_idx, src_dir, dst_dir, label):
    for i in range(start_idx, end_idx + 1):
        file_name = f'{label}.{i}.jpg'
        src_path = os.path.join(src_dir, file_name)
        dst_path = os.path.join(dst_dir, file_name)
        if os.path.exists(src_path):
            shutil.copy(src_path, dst_path)

# 클래스별로 train, valid, test 데이터셋 구성
for cls in classes:
    # train dataset 구성 (0~999)
    copy_images(train_range[0], train_range[1], dataset_dir, os.path.join(train_dir, cls), cls)

    # valid dataset 구성 (1000~1249)
    copy_images(valid_range[0], valid_range[1], dataset_dir, os.path.join(valid_dir, cls), cls)

    # test dataset 구성 (1250~1499)
    copy_images(test_range[0], test_range[1], dataset_dir, os.path.join(test_dir), cls)

print("데이터셋 분할 및 복사가 완료되었습니다.")

데이터셋 분할 및 복사가 완료되었습니다.


In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 88.4 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
import timm  # PyTorch Image Models (timm) 라이브러리
import os
from PIL import Image
from torch.utils.data import Dataset

In [ ]:
!pip install pytorch-ignite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.7/312.7 kB 19.0 MB/s eta 0:00:00


In [ ]:
# Kaggle의 Dogs vs. Cats 데이터셋을 위한 CustomDataset 클래스 생성
class DogsVsCatsDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# 파일 경로와 레이블을 생성하는 함수
def get_file_paths_and_labels(dataset_dir):
    file_paths = []
    labels = []

    for label, class_name in enumerate(['cat', 'dog']):
        class_dir = os.path.join(dataset_dir, class_name)
        for file_name in os.listdir(class_dir):
            file_paths.append(os.path.join(class_dir, file_name))
            labels.append(label)  # cat=0, dog=1

    return file_paths, labels

In [ ]:
# 데이터셋 디렉토리 경로 설정 (Kaggle 데이터셋 경로 설정)
train_dir = 'sdata/train'  # train 데이터셋 경로
valid_dir = 'sdata/valid'  # valid 데이터셋 경로

# 학습 데이터셋 및 검증 데이터셋에 대한 파일 경로 및 레이블 생성
train_file_paths, train_labels = get_file_paths_and_labels(train_dir)
valid_file_paths, valid_labels = get_file_paths_and_labels(valid_dir)

# 이미지 전처리 설정
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 데이터셋 준비
train_dataset = DogsVsCatsDataset(train_file_paths, train_labels, transform=train_transform)
valid_dataset = DogsVsCatsDataset(valid_file_paths, valid_labels, transform=valid_transform)

# 데이터 로더 준비
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# EfficientNetV2B0 모델 불러오기 (timm 라이브러리 사용)
model = timm.create_model('efficientnet_b0', pretrained=True)

# 마지막 레이어의 입력 특징 수를 가져옵니다.
num_features = model.classifier.in_features

# 이진 분류를 위한 새로운 분류기 추가 (마지막 레이어 수정)
model.classifier = nn.Sequential(
    nn.Linear(num_features, 1),  # 출력 노드를 1개로 설정 (이진 분류)
    nn.Sigmoid()  # Sigmoid 활성화 함수 추가
)

# 모델을 GPU 또는 CPU에 할당
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 손실 함수와 최적화 알고리즘 설정
criterion = nn.BCELoss()  # 이진 크로스 엔트로피 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.0001)
# 학습율을 조정
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

In [ ]:
# EarlyStopping 클래스
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): 성능 개선이 없을 때 몇 번의 에포크까지 기다릴지.
            verbose (bool): True일 경우 개선될 때마다 메시지 출력.
            delta (float): 성능 개선으로 간주될 최소 변화량.
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss
        # 처음에 호출됐을때는 best_score가 None이라서 초기값을 설정
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        # 지금까지의 best_score와 현재 score를 비교
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            # patience값이 모두 충족했을때, 종료조건이 만족될때
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''검증 손실이 감소하면 모델을 저장합니다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
            torch.save(model.state_dict(), self.path)  # 모델 상태 저장
        self.val_loss_min = val_loss

# EarlyStopping 인스턴스 생성 (patience=10)
early_stopping = EarlyStopping(patience=10, verbose=True, path='efficientnet_b0_best.pth')

In [ ]:
# 모델 학습 함수
def train_model(model, criterion, optimizer, train_loader, valid_loader, epochs):
    # 훈련 시작시 초기 lr을 로딩
    pre_lr = optimizer.param_groups[0]['lr']

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0

        # Training
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device).float()

            # 모델 초기화
            optimizer.zero_grad()

            # Forward propagation
            outputs = model(inputs).squeeze()  # (batch_size, 1) -> (batch_size,)
            loss = criterion(outputs, labels)

            # Backward propagation
            loss.backward()
            optimizer.step()

            # 손실과 정확도 계산
            running_loss += loss.item() * inputs.size(0)
            preds = (outputs >= 0.5).float()  # 0.5 이상은 1, 이하 0으로 이진화
            correct += torch.sum(preds == labels)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct.double() / len(train_loader.dataset)

        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device).float()

                outputs = model(inputs).squeeze()
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                preds = (outputs >= 0.5).float()
                val_correct += torch.sum(preds == labels)

        val_epoch_loss = val_loss / len(valid_loader.dataset)
        val_epoch_acc = val_correct.double() / len(valid_loader.dataset)

        print(f'valid_loss: {val_epoch_loss:.4f}, valid_acc: {val_epoch_acc:.4f}')

        # ReduceLROnPlateau 스케줄러를 사용하여 검증 손실에 따라 학습률을 조정
        scheduler.step(val_epoch_loss)

        # 현재 학습률 출력
        now_lr = optimizer.param_groups[0]['lr']
        if now_lr != pre_lr:
            pre_lr = now_lr
            lr_str = ', LR changed!!'
        else:
            lr_str = ''

        print(f'learning_rate {epoch+1}: {now_lr:.8f}'+lr_str)

        # EarlyStopping을 호출하여 학습 중단 여부 확인
        early_stopping(val_epoch_loss, model)

        # 학습 중단 조건을 충족하면 break
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break
        print('-' * 70)

In [ ]:
# 모델 학습
train_model(model, criterion, optimizer, train_loader, valid_loader, epochs=30)

Epoch 1/30, Loss: 0.3135, Accuracy: 0.9110
valid_loss: 0.0714, valid_acc: 0.9780
learning_rate 1: 0.00010000
Validation loss decreased (inf --> 0.071388).  Saving model ...
----------------------------------------------------------------------
Epoch 2/30, Loss: 0.0519, Accuracy: 0.9890
valid_loss: 0.0510, valid_acc: 0.9810
learning_rate 2: 0.00010000
Validation loss decreased (0.071388 --> 0.050960).  Saving model ...
----------------------------------------------------------------------
Epoch 3/30, Loss: 0.0282, Accuracy: 0.9930
valid_loss: 0.0414, valid_acc: 0.9870
learning_rate 3: 0.00010000
Validation loss decreased (0.050960 --> 0.041408).  Saving model ...
----------------------------------------------------------------------
Epoch 4/30, Loss: 0.0097, Accuracy: 0.9990
valid_loss: 0.0399, valid_acc: 0.9870
learning_rate 4: 0.00010000
Validation loss decreased (0.041408 --> 0.039936).  Saving model ...
----------------------------------------------------------------------
Epoch 5/3

In [7]:
from google.colab import drive
drive.mount('/content/drive')

import os
modelPath = '/content/drive/MyDrive/model'
os.makedirs(modelPath, exist_ok=True)

Mounted at /content/drive


In [ ]:
modelPath = '/content/drive/MyDrive/model'
modelname = 'efficientnet_b0.pth'

torch.save(model.state_dict(), os.path.join(modelPath, modelname))

In [ ]:
# 모델에 저장된 가중치 로드
model.load_state_dict(torch.load(os.path.join(modelPath, modelname)))

<ipython-input-18-32a5c037ceab>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(modelPath, modelname)))


<All keys matched successfully>

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# EfficientNetV2B0 모델 불러오기 (timm 라이브러리 사용)
model = timm.create_model('efficientnet_b0', pretrained=True)

# EfficientNet-B0의 특징 추출기(frozen)를 고정시키기
for param in model.parameters():
    param.requires_grad = False  # 모든 파라미터 고정

# 마지막 레이어의 입력 특징 수를 가져옵니다.
num_features = model.classifier.in_features

# 이진 분류를 위한 새로운 분류기 추가 (마지막 레이어 수정)
model.classifier = nn.Sequential(
    torch.nn.Linear(num_features, 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(256, 1),
    torch.nn.Sigmoid()
)

# 분류기 부분만 학습되도록 파라미터 업데이트 설정
for param in model.classifier.parameters():
    param.requires_grad = True  # 분류기 부분만 학습 가능하게 설정

# 모델을 GPU 또는 CPU에 할당
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 손실 함수와 최적화 알고리즘 설정
criterion = nn.BCELoss()  # 이진 크로스 엔트로피 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
early_stopping = EarlyStopping(patience=10, verbose=True, path='efficientnet_b0_best.pth')

In [ ]:
train_model(model, criterion, optimizer, train_loader, valid_loader, epochs=5)

Epoch 1/5, Loss: 0.5540, Accuracy: 0.8125
valid_loss: 0.3977, valid_acc: 0.9310
learning_rate 1: 0.00010000
Validation loss decreased (inf --> 0.397747).  Saving model ...
----------------------------------------------------------------------
Epoch 2/5, Loss: 0.3107, Accuracy: 0.9180
valid_loss: 0.2359, valid_acc: 0.9550
learning_rate 2: 0.00010000
Validation loss decreased (0.397747 --> 0.235873).  Saving model ...
----------------------------------------------------------------------
Epoch 3/5, Loss: 0.2179, Accuracy: 0.9425
valid_loss: 0.1829, valid_acc: 0.9630
learning_rate 3: 0.00010000
Validation loss decreased (0.235873 --> 0.182864).  Saving model ...
----------------------------------------------------------------------
Epoch 4/5, Loss: 0.1696, Accuracy: 0.9500
valid_loss: 0.1503, valid_acc: 0.9690
learning_rate 4: 0.00010000
Validation loss decreased (0.182864 --> 0.150334).  Saving model ...
----------------------------------------------------------------------
Epoch 5/5, Lo

In [ ]:
model

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (aa): Identity()
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2

In [ ]:
def display_parameters(model):
    # 학습 가능한 파라미터의 개수 출력
    num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Number of trainable parameters: {num_trainable_params}")

In [ ]:
for param in model.classifier.parameters():
    param.requires_grad = False  # 분류기 부분만 학습 가능하게 설정

display_parameters(model)

In [ ]:
# 마지막 두 블록을 가져와서 requires_grad를 True로 설정
for param in model.blocks[-2:].parameters():
    param.requires_grad = True

In [ ]:
# 특정 레이어 이름을 기준으로 동결 해제
for name, param in model.named_parameters():
    if "blocks.5" in name or "blocks.6" in name:  # 예를 들어, 마지막 두 블록
        param.requires_grad = True

display_parameters(model)

Number of trainable parameters: 2743580


In [ ]:
for name, param in model.named_parameters():
    print(name)

conv_stem.weight
bn1.weight
bn1.bias
blocks.0.0.conv_dw.weight
blocks.0.0.bn1.weight
blocks.0.0.bn1.bias
blocks.0.0.se.conv_reduce.weight
blocks.0.0.se.conv_reduce.bias
blocks.0.0.se.conv_expand.weight
blocks.0.0.se.conv_expand.bias
blocks.0.0.conv_pw.weight
blocks.0.0.bn2.weight
blocks.0.0.bn2.bias
blocks.1.0.conv_pw.weight
blocks.1.0.bn1.weight
blocks.1.0.bn1.bias
blocks.1.0.conv_dw.weight
blocks.1.0.bn2.weight
blocks.1.0.bn2.bias
blocks.1.0.se.conv_reduce.weight
blocks.1.0.se.conv_reduce.bias
blocks.1.0.se.conv_expand.weight
blocks.1.0.se.conv_expand.bias
blocks.1.0.conv_pwl.weight
blocks.1.0.bn3.weight
blocks.1.0.bn3.bias
blocks.1.1.conv_pw.weight
blocks.1.1.bn1.weight
blocks.1.1.bn1.bias
blocks.1.1.conv_dw.weight
blocks.1.1.bn2.weight
blocks.1.1.bn2.bias
blocks.1.1.se.conv_reduce.weight
blocks.1.1.se.conv_reduce.bias
blocks.1.1.se.conv_expand.weight
blocks.1.1.se.conv_expand.bias
blocks.1.1.conv_pwl.weight
blocks.1.1.bn3.weight
blocks.1.1.bn3.bias
blocks.2.0.conv_pw.weight
blocks.